In [106]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")
#pd.set_option('display.float_format', '{:.2f}'.format)

In [107]:
test = pd.read_csv('Test_TP2_Datos_2020-2C.csv')
train = pd.read_csv('Train_TP2_Datos_2020-2C.csv')

In [208]:
#train.info()

In [209]:
#test.info()

* Contamos con un total de:
> * 16947 Registros en nuestro set de datos de *train*. 
> * 2551 Registros en nuestro set de datos de *test*.
> * Los 2551 registros a predecir en test, representan un 15% en cantidad de registros que tenemos en train (16947). En cantidad de oportunidades como son los porcentajes?

* Cuántas Oportunidades distintas tengo en *train* y *test*?

In [112]:
len(set(train.Opportunity_Name))

9841

In [113]:
len(set(test.Opportunity_Name))

1567

> * Se observa que las oportunidades a predecir en el set de *test* representan un 15,9% en cantidad de oportunidades que tenemos en *train*, manteniendo casi la misma proporción que teníamos con la cantidad de registros.

 * Veamos si existen registros duplicadas y para eso comparamos todos los valores existentes cada una de las filas sin tener en cuenta la columna *ID* que es el id único del registro o fila.

In [114]:
train[train.duplicated(subset=list(train.columns)[1:],keep=False)].shape[0]

619

In [115]:
test[test.duplicated(subset=list(test.columns)[1:],keep=False)].shape[0]

187

* Tenemos 619 Registros de *train* y 187 Registros de *test* que se repiten en nuestro set de datos.

* Respecto a esto: en nuestro set de *test* no podemos eliminar duplicados ya que son a todos esos registros a los que debemos predecir el *Stage*. En cambio en *train* podría pasar que filtrando estos casos repetidos podamos llegar a tener una mejor performance en nuestras predicciones ya que estaré entrenando con registros distintos y saco duplicados que no aportarían a priori información al entrenamiento sino mas bien algo de ruido. (Probar y en el Informe comentar si hay alguna mejora filtrando duplicados)

* Si nos quedamos en *train* solo con la primera aparición de esos Registros duplicados, cuantos Registros en total debería filtrar?

In [116]:
train[train.duplicated(subset=list(train.columns)[1:],keep='first')].shape[0]

448

* A cuántas oportunidades distintas corresponden esos registros duplicados?

In [117]:
len(set(train[train.duplicated(subset=list(train.columns)[1:],keep=False)].Opportunity_Name))

131

In [118]:
# Vamos a eliminar duplicados excepto los Registros que correspondad a la primera aparición.
#train = train.drop_duplicates(subset=list(train.columns)[1:],keep='first') 
#len(set(train.ID))

* Si realizamos el filtrado de Registros repetidos en *train*, conservamos las mismas cantidades de Oportunidades (9841) pero pasamos de 16947 a 16499 Registros, osea que eliminamos 448 Registros duplicados que corresponden a un 2,64% de los registros iniciales. (Probar performance con este filtro)

### Limpieza de Regiones y Territorios 
##### (*la limpieza aplicada en train debe ser la misma que en test*)

In [119]:
train.Region.value_counts()

Japan          4892
EMEA           4664
Americas       3945
APAC           3262
Middle East     184
Name: Region, dtype: int64

In [120]:
test.Region.value_counts()

Americas       1016
EMEA            838
APAC            482
Japan           201
Middle East      14
Name: Region, dtype: int64

In [121]:
# Para los casos en que tenga a Japon como Region y a None como Territory le asigno Japón a Territory, sinó lo dejo como está
train['Territory'] = train.apply(lambda row: row['Region'] if ((row['Territory']=='None') & (row['Region']=='Japan')) else row['Territory'],axis=1)
test['Territory'] = test.apply(lambda row: row['Region'] if ((row['Territory']=='None') & (row['Region']=='Japan')) else row['Territory'],axis=1)

In [122]:
# Dejo Japón solo como territorio y como Región le asigno APAC (Asia-Pacífico)
train.loc[train['Region']=='Japan','Region']='Asia-Pacífico'
test.loc[test['Region']=='Japan','Region']='Asia-Pacífico'

In [123]:
# Cambio el nombre de la región Americas por América
train.loc[train['Region']=='Americas','Region']='América'
# Cambio el nombre de las demás Regiones
train.loc[train['Region']=='EMEA','Region']='Europa'
train.loc[train['Region']=='Middle East','Region']='Oriente Medio'
train.loc[train['Region']=='APAC','Region']='Asia-Pacífico'

# Lo mismo con test
test.loc[test['Region']=='Americas','Region']='América'
# Cambio el nombre de las demás Regiones
test.loc[test['Region']=='EMEA','Region']='Europa'
test.loc[test['Region']=='Middle East','Region']='Oriente Medio'
test.loc[test['Region']=='APAC','Region']='Asia-Pacífico'

In [124]:
train.Region.value_counts()

Asia-Pacífico    8154
Europa           4664
América          3945
Oriente Medio     184
Name: Region, dtype: int64

In [125]:
test.Region.value_counts()

América          1016
Europa            838
Asia-Pacífico     683
Oriente Medio      14
Name: Region, dtype: int64

* Un dato interesante a destacar es que en train la región predominante es Asia-Pacífico y en test en América siendo Asia-Pacífico la que ocupa la tercera posición. Este dato no es menor ya que puede que según cada región puedo sectorizar los entrenemientos y predicciones. Vamos a probar esta segmentación de predicciones para ver si obtenemos mejor performance que entrenar todo junto.

In [126]:
## Arreglamos la asignación de los territorios a sus correspondientes Regiones
train.loc[train['Territory']=='Saudi Arabia','Territory']= 'Arabia Saudita'
train.loc[train['Territory']=='KSA','Territory']= 'Arabia Saudita'
train.loc[(train['Territory']=='Arabia Saudita')|(train['Territory']=='Jordan')|(train['Territory']=='UAE (Dubai)')\
       |(train['Territory']=='Egypt')|(train['Territory']=='Kuwait'),'Region']= 'Oriente Medio'
train.loc[(train['Territory']=='South America')|(train['Territory']=='SE America')|(train['Territory']=='SW America')\
       |(train['Territory']=='Central America'),'Region']= 'América'
train.loc[train['Territory']=='Kazakhstan','Region']= 'Asia-Pacífico'

## Lo mismo para test
test.loc[test['Territory']=='Saudi Arabia','Territory']= 'Arabia Saudita'
test.loc[test['Territory']=='KSA','Territory']= 'Arabia Saudita'
test.loc[(test['Territory']=='Arabia Saudita')|(test['Territory']=='Jordan')|(test['Territory']=='UAE (Dubai)')\
       |(test['Territory']=='Egypt')|(test['Territory']=='Kuwait'),'Region']= 'Oriente Medio'
test.loc[(test['Territory']=='South America')|(test['Territory']=='SE America')|(test['Territory']=='SW America')\
       |(test['Territory']=='Central America'),'Region']= 'América'
test.loc[test['Territory']=='Kazakhstan','Region']= 'Asia-Pacífico'

### Agregamos el Hemisferio según corresponda a cada Territorio.

In [127]:
# Voy a Clasificar los Territorios según los Hemisferios que pertenecen
mis_kw_train = train['Territory']
mis_kw_test = test['Territory']

from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()
keyword_dict = {"Hemisferio Norte": ["NW America","Japan", "India","Italy","Germany","United Kingdom","UAE (Dubai)","Greece"\
                                     ,"Switzerland","Austria","NE America","Belgium","Tunisia",'SE America',"France","Spain","Singapore"\
                                     ,"Philippines","SW America",'Denmark', 'Central America', 'Poland', 'Norway', 'Thailand'\
                                     ,'Portugal', 'South Korea','Netherlands','Kuwait','Arabia Saudita','Cambodia', 'China (PRC)', 'KSA', 'Ireland'\
                                     , 'Jordan','Cyprus', 'Morocco', 'Turkey', 'Sweden', 'Monaco', 'Kazakhstan','Romania', 'Hungary'\
                                     , 'Taiwan','Macedonia', 'Ukraine', 'Czech Republic', 'Belarus','Vietnam', 'Saudi Arabia'\
                                     ,'Bosnia & Herzegovina', 'Kosovo', 'Luxembourg', 'Bulgaria','UAE (Others)', 'Armenia', 'Croatia'\
                                     , 'Malaysia', 'Georgia','Estonia', 'Pakistan', 'Canada', 'Central US', 'Carribean','Malta'\
                                     , 'Albania', 'Oman', 'Burma', 'Slovenia','Egypt','None','Andorra','Finland','Mongolia']\
               ,"Hemisferio Sur": ["Australia",'South America','Indonesia','South East Asia','Solomon Islands','New Zealand']\
             }
key_depuradas_train=[]
key_depuradas_test=[]
keyword_processor.add_keywords_from_dict(keyword_dict)

for w in mis_kw_train:
       if len(keyword_processor.extract_keywords(w))==1:
            key_depuradas_train.append(keyword_processor.extract_keywords(w)[0])
       
       else: 
            key_depuradas_train.append(w)
            
for w in mis_kw_test:
       if len(keyword_processor.extract_keywords(w))==1:
            key_depuradas_test.append(keyword_processor.extract_keywords(w)[0])
       
       else: 
            key_depuradas_test.append(w)     

In [128]:
len(key_depuradas_train)

16947

In [129]:
len(key_depuradas_test)

2551

In [130]:
len(train['Territory'])

16947

In [131]:
len(test['Territory'])

2551

In [132]:
train['Hemisferio'] = key_depuradas_train
test['Hemisferio'] = key_depuradas_test

In [133]:
### Analizar hemisferios
#¿Filtro los hemisferios para los cuales no tenía un territorio definido?
#df = df.loc[(df['Hemisferio']!='None'),:]

In [134]:
train[['ID','Region','Territory','Hemisferio']].tail()

,ID,Region,Territory,Hemisferio
16942,8781,Europa,Austria,Hemisferio Norte
16943,8786,Europa,Austria,Hemisferio Norte
16944,8792,Europa,Austria,Hemisferio Norte
16945,28561,América,NE America,Hemisferio Norte
16946,28318,América,NE America,Hemisferio Norte


In [135]:
test[['ID','Region','Territory','Hemisferio']].tail()

,ID,Region,Territory,Hemisferio
2546,16345,Oriente Medio,Arabia Saudita,Hemisferio Norte
2547,15218,América,SE America,Hemisferio Norte
2548,15224,América,SE America,Hemisferio Norte
2549,7286,América,NE America,Hemisferio Norte
2550,11263,Europa,Belgium,Hemisferio Norte


In [210]:
#set(test.Territory)

In [211]:
#set(train.Territory)

### Agregamos La estación del año en que se crean y se entregan las oportunidades en funcion del Hemisferio del Terrirorio al que pertenecen.

> * (podría agregar estaciones de entrega o de ultima modificación para tener más features)

In [138]:
train['Opportunity_Created_Date']=pd.to_datetime(train['Opportunity_Created_Date'])
test['Opportunity_Created_Date']=pd.to_datetime(test['Opportunity_Created_Date'])
train['Planned_Delivery_Start_Date']=pd.to_datetime(train['Planned_Delivery_Start_Date'])
test['Planned_Delivery_Start_Date']=pd.to_datetime(test['Planned_Delivery_Start_Date'])

In [139]:
def encuentra_Estacion_N(date): 
    year = str(date.year)
    seasons = {'Primavera': pd.date_range(start='21/03/'+year, end='20/06/'+year),
               'Verano': pd.date_range(start='21/06/'+year, end='20/09/'+year),
               'Otono': pd.date_range(start='21/09/'+year, end='20/12/'+year)}
    if date in seasons['Primavera']:
        return 'Primavera'
    if date in seasons['Verano']:
        return 'Verano'
    if date in seasons['Otono']:
        return 'Otono'
    else:
        return 'Invierno'   
    
def encuentra_Estacion_S(date): 
    year = str(date.year)
    seasons = {'Otono': pd.date_range(start='21/03/'+year, end='20/06/'+year),
               'Invierno': pd.date_range(start='21/06/'+year, end='20/09/'+year),
               'Primavera': pd.date_range(start='21/09/'+year, end='20/12/'+year)}
    if date in seasons['Invierno']:
        return 'Invierno'
    if date in seasons['Primavera']:
        return 'Primavera'
    if date in seasons['Otono']:
        return 'Otono'
    else:
        return 'Verano'     

In [140]:
train.loc[train['Hemisferio']=='Hemisferio Norte','Estación'] = train.loc[train['Hemisferio']=='Hemisferio Norte','Opportunity_Created_Date']\
.map(encuentra_Estacion_N)
train.loc[train['Hemisferio']=='Hemisferio Sur','Estación'] = train.loc[train['Hemisferio']=='Hemisferio Sur','Opportunity_Created_Date']\
.map(encuentra_Estacion_S)

test.loc[test['Hemisferio']=='Hemisferio Norte','Estación'] = test.loc[test['Hemisferio']=='Hemisferio Norte','Opportunity_Created_Date']\
.map(encuentra_Estacion_N)
test.loc[test['Hemisferio']=='Hemisferio Sur','Estación'] = test.loc[test['Hemisferio']=='Hemisferio Sur','Opportunity_Created_Date']\
.map(encuentra_Estacion_S)


train.loc[train['Hemisferio']=='Hemisferio Norte','Estacion_entrega'] = train.loc[train['Hemisferio']=='Hemisferio Norte','Planned_Delivery_Start_Date']\
.map(encuentra_Estacion_N)
train.loc[train['Hemisferio']=='Hemisferio Sur','Estacion_entrega'] = train.loc[train['Hemisferio']=='Hemisferio Sur','Planned_Delivery_Start_Date']\
.map(encuentra_Estacion_S)

test.loc[test['Hemisferio']=='Hemisferio Norte','Estacion_entrega'] = test.loc[test['Hemisferio']=='Hemisferio Norte','Planned_Delivery_Start_Date']\
.map(encuentra_Estacion_N)
test.loc[test['Hemisferio']=='Hemisferio Sur','Estacion_entrega'] = test.loc[test['Hemisferio']=='Hemisferio Sur','Planned_Delivery_Start_Date']\
.map(encuentra_Estacion_S)

In [141]:
train[['Opportunity_ID','Opportunity_Created_Date','Hemisferio','Estación','Estacion_entrega']].sample(10)

,Opportunity_ID,Opportunity_Created_Date,Hemisferio,Estación,Estacion_entrega
2706,1498,2016-04-19,Hemisferio Sur,Otono,Invierno
6954,4284,2017-05-09,Hemisferio Norte,Primavera,Primavera
16649,12654,2015-10-09,Hemisferio Norte,Otono,Primavera
8592,5374,2017-06-29,Hemisferio Norte,Verano,Verano
11754,7183,2018-03-29,Hemisferio Sur,Otono,Invierno
16804,12728,2015-10-21,Hemisferio Norte,Otono,Verano
12079,7363,2018-01-10,Hemisferio Sur,Verano,Otono
15915,10388,2018-11-29,Hemisferio Norte,Otono,Otono
15986,10537,2018-12-07,Hemisferio Norte,Otono,Otono
13457,8184,2018-05-10,Hemisferio Norte,Primavera,Verano


In [212]:
#train.info()

In [143]:
train.Estación.value_counts(normalize=True)

Otono        0.270522
Primavera    0.265447
Verano       0.233992
Invierno     0.230038
Name: Estación, dtype: float64

In [144]:
test.Estación.value_counts(normalize=True)

Invierno     0.684829
Primavera    0.226970
Otono        0.051352
Verano       0.036848
Name: Estación, dtype: float64

In [145]:
train.shape

(16947, 55)

### Precio de las oportunidades

* Vamos a crear un feature con el valor total de ventas de las oportunidades (**Total_Taxable_Amount**) en una misma moneda (**dólares estadounidenses**) para que la comparación entre valores sean coherentes. También cabe destacar que en *test* tenemos casos de **Total_Taxable_Amount = 0** pero no tenemos casos de **Total_Amount = 0**, con lo cual calcularemos el precio final de cada **oportunidad** sumando dichos valores de Total_Amount para cada una de ellas. Esto lo vamos a realizar también en *train* pero tener en cuenta que train además cuenta con registros que contienen simultaneamente **Total_Taxable_Amount y Total_Amount** igual a 0, (específicamente 39 casos que no se dan en test) con lo cual debo tomar la mejor decición, ya sea filtrando esos casos de train o imputando algún valor a Total_Taxable_Amount.    

In [146]:
train.loc[(train['Total_Amount']==0) & ((train['Total_Taxable_Amount']==0)),:]

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Hemisferio,Estación,Estacion_entrega
464,28260,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/30/2015,...,0,USD,0.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,Primavera
579,25679,Asia-Pacífico,Japan,1,0,0,0,0,Bureaucratic_Code_5,4/20/2015,...,0,USD,0.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otono,Primavera
580,25718,Asia-Pacífico,Japan,1,0,0,0,0,Bureaucratic_Code_5,4/20/2015,...,0,USD,0.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otono,Primavera
586,15710,Asia-Pacífico,Japan,1,0,0,0,0,Bureaucratic_Code_5,12/9/2016,...,0,USD,0.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Otono,Verano
1130,17845,Europa,Germany,1,0,1,0,0,Bureaucratic_Code_5,10/29/2013,...,0,EUR,0.0,EUR,0.0,Closed Won,Prod_Category_A_None,Hemisferio Norte,Invierno,Primavera
1437,22806,América,NW America,1,0,1,0,0,Bureaucratic_Code_5,7/27/2013,...,0,USD,0.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,Verano
1705,14772,Oriente Medio,None,1,0,1,0,0,Bureaucratic_Code_5,11/15/2016,...,0,USD,0.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,Primavera
1904,8897,Europa,United Kingdom,1,0,1,0,0,Bureaucratic_Code_5,1/20/2016,...,5,EUR,0.0,EUR,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,Primavera
2083,10670,América,SE America,1,1,1,1,0,Bureaucratic_Code_4,2/13/2016,...,0,USD,0.0,USD,0.0,Closed Won,Prod_Category_A_None,Hemisferio Norte,Invierno,Invierno
2823,14860,Europa,None,1,0,1,0,0,Bureaucratic_Code_5,5/9/2016,...,5,USD,0.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Primavera,Verano


In [147]:
train = train.loc[(train['Total_Amount']!=0) | ((train['Total_Taxable_Amount']!=0)),:]

In [148]:
train.shape

(16908, 55)

In [149]:
test.loc[test['Total_Amount']==0,:]

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A,Hemisferio,Estación,Estacion_entrega


In [150]:
test.loc[(test['Total_Amount']==0) & ((test['Total_Taxable_Amount']==0)),:]

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A,Hemisferio,Estación,Estacion_entrega


In [151]:
train.iloc[186:188]

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Hemisferio,Estación,Estacion_entrega
186,8142,América,NE America,1,1,1,1,0,Bureaucratic_Code_4,12/24/2015,...,1,USD,659835.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,Primavera
187,8148,América,NE America,1,1,1,1,0,Bureaucratic_Code_4,12/24/2015,...,1,USD,672512.5,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,Primavera


In [152]:
train_monto_total=train.groupby('Opportunity_ID').agg({'Total_Amount':'sum'}).reset_index()
train_monto_total.rename(columns={'Total_Amount':'Total_moneda_local'},inplace=True)
train_monto_total.head()

,Opportunity_ID,Total_moneda_local
0,0,5272800.0
1,1,48230.0
2,2,83865.6
3,3,7421881.5
4,4,13357192.5


In [153]:
test_monto_total=test.groupby('Opportunity_ID').agg({'Total_Amount':'sum'}).reset_index()
test_monto_total.rename(columns={'Total_Amount':'Total_moneda_local'},inplace=True)
test_monto_total.head()

,Opportunity_ID,Total_moneda_local
0,10689,367419.0
1,10690,757783.5
2,10691,21037.5
3,10692,2169106.5
4,10693,5752.5


In [154]:
train=pd.merge(train,train_monto_total,how='inner',on='Opportunity_ID')

In [155]:
test=pd.merge(test,test_monto_total,how='inner',on='Opportunity_ID')

In [156]:
train.iloc[186:188]

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,...,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A,Hemisferio,Estación,Estacion_entrega,Total_moneda_local
186,8142,América,NE America,1,1,1,1,0,Bureaucratic_Code_4,12/24/2015,...,USD,659835.0,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,Primavera,1332347.5
187,8148,América,NE America,1,1,1,1,0,Bureaucratic_Code_4,12/24/2015,...,USD,672512.5,USD,0.0,Closed Lost,Prod_Category_A_None,Hemisferio Norte,Invierno,Primavera,1332347.5


In [157]:
train.ASP_Currency.value_counts()

USD    7338
JPY    4520
EUR    4489
AUD     556
GBP       5
Name: ASP_Currency, dtype: int64

In [213]:
#train.loc[train['ASP_Currency']=='GBP',:]

In [160]:
def conver_EUR_dolares(monto): 
        return monto*1.131   
    
def conver_JPY_dolares(monto): 
        return monto*0.00899 

def conver_AUD_dolares(monto): 
        return monto*0.7076 
    
def conver_GBP_dolares(monto): 
        return monto*1.3181 

def conver_USD_dolares(monto): 
        return monto*1

In [161]:
train.loc[train['Total_Taxable_Amount_Currency']=='EUR','total_en_dolares'] = train.loc[train['Total_Taxable_Amount_Currency']=='EUR','Total_moneda_local']\
.map(conver_EUR_dolares)
train.loc[train['Total_Taxable_Amount_Currency']=='JPY','total_en_dolares'] = train.loc[train['Total_Taxable_Amount_Currency']=='JPY','Total_moneda_local']\
.map(conver_JPY_dolares)
train.loc[train['Total_Taxable_Amount_Currency']=='AUD','total_en_dolares'] = train.loc[train['Total_Taxable_Amount_Currency']=='AUD','Total_moneda_local']\
.map(conver_AUD_dolares)
train.loc[train['Total_Taxable_Amount_Currency']=='GBP','total_en_dolares'] = train.loc[train['Total_Taxable_Amount_Currency']=='GBP','Total_moneda_local']\
.map(conver_GBP_dolares)
train.loc[train['Total_Taxable_Amount_Currency']=='USD','total_en_dolares'] = train.loc[train['Total_Taxable_Amount_Currency']=='USD','Total_moneda_local']\
.map(conver_USD_dolares)

In [162]:
test.loc[test['Total_Taxable_Amount_Currency']=='EUR','total_en_dolares'] = test.loc[test['Total_Taxable_Amount_Currency']=='EUR','Total_moneda_local']\
.map(conver_EUR_dolares)
test.loc[test['Total_Taxable_Amount_Currency']=='JPY','total_en_dolares'] = test.loc[test['Total_Taxable_Amount_Currency']=='JPY','Total_moneda_local']\
.map(conver_JPY_dolares)
test.loc[test['Total_Taxable_Amount_Currency']=='AUD','total_en_dolares'] = test.loc[test['Total_Taxable_Amount_Currency']=='AUD','Total_moneda_local']\
.map(conver_AUD_dolares)
test.loc[test['Total_Taxable_Amount_Currency']=='GBP','total_en_dolares'] = test.loc[test['Total_Taxable_Amount_Currency']=='GBP','Total_moneda_local']\
.map(conver_GBP_dolares)
test.loc[test['Total_Taxable_Amount_Currency']=='USD','total_en_dolares'] = test.loc[test['Total_Taxable_Amount_Currency']=='USD','Total_moneda_local']\
.map(conver_USD_dolares)

### Total Amount en dólares

In [163]:
train.loc[train['Total_Amount_Currency']=='EUR','monto_en_dolares'] = train.loc[train['Total_Amount_Currency']=='EUR','Total_Amount']\
.map(conver_EUR_dolares)
train.loc[train['Total_Amount_Currency']=='JPY','monto_en_dolares'] = train.loc[train['Total_Amount_Currency']=='JPY','Total_Amount']\
.map(conver_JPY_dolares)
train.loc[train['Total_Amount_Currency']=='AUD','monto_en_dolares'] = train.loc[train['Total_Amount_Currency']=='AUD','Total_Amount']\
.map(conver_AUD_dolares)
train.loc[train['Total_Amount_Currency']=='GBP','monto_en_dolares'] = train.loc[train['Total_Amount_Currency']=='GBP','Total_Amount']\
.map(conver_GBP_dolares)
train.loc[train['Total_Amount_Currency']=='USD','monto_en_dolares'] = train.loc[train['Total_Amount_Currency']=='USD','Total_Amount']\
.map(conver_USD_dolares)

test.loc[test['Total_Amount_Currency']=='EUR','monto_en_dolares'] = test.loc[test['Total_Amount_Currency']=='EUR','Total_Amount']\
.map(conver_EUR_dolares)
test.loc[test['Total_Amount_Currency']=='JPY','monto_en_dolares'] = test.loc[test['Total_Amount_Currency']=='JPY','Total_Amount']\
.map(conver_JPY_dolares)
test.loc[test['Total_Amount_Currency']=='AUD','monto_en_dolares'] = test.loc[test['Total_Amount_Currency']=='AUD','Total_Amount']\
.map(conver_AUD_dolares)
test.loc[test['Total_Amount_Currency']=='GBP','monto_en_dolares'] = test.loc[test['Total_Amount_Currency']=='GBP','Total_Amount']\
.map(conver_GBP_dolares)
test.loc[test['Total_Amount_Currency']=='USD','monto_en_dolares'] = test.loc[test['Total_Amount_Currency']=='USD','Total_Amount']\
.map(conver_USD_dolares)

In [168]:
train.groupby('Opportunity_ID').agg({'monto_en_dolares':['max','min','mean','std']}).reset_index()

Opportunity_ID monto_en_dolares                              \
                                 max           min          mean   
0                 0       5963536.80  5.963537e+06  5.963537e+06   
1                 1         54548.13  5.454813e+04  5.454813e+04   
2                 2         83865.60  8.386560e+04  8.386560e+04   
3                 3       7421881.50  7.421882e+06  7.421882e+06   
4                 4      13357192.50  1.335719e+07  1.335719e+07   
...             ...              ...           ...           ...   
9800          12799        454322.70  4.543227e+05  4.543227e+05   
9801          12800      10751580.00  1.058092e+07  1.066625e+07   
9802          12801        116888.85  1.052000e+05  1.129926e+05   
9803          12802       2346796.88  2.346797e+06  2.346797e+06   
9804          12803      25603200.00  2.560320e+07  2.560320e+07   

                     
                std  
0               NaN  
1               NaN  
2               NaN  
3               NaN  
4               NaN  
...             ...  
9800            NaN  
9801  120674.843277  
9802    6748.580901  
9803            NaN  
9804            NaN  

[9805 rows x 5 columns]

In [169]:
test.total_en_dolares=test.total_en_dolares.astype('float')
test.monto_en_dolares=test.monto_en_dolares.astype('float')

In [170]:
train_monto=train.groupby('Opportunity_ID').agg({'monto_en_dolares':['max','min','mean','std']}).reset_index()
test_monto=test.groupby('Opportunity_ID').agg({'monto_en_dolares':['max','min','mean','std']}).reset_index()

In [171]:
train_monto.sample(5)

Opportunity_ID monto_en_dolares                                          
                                 max           min          mean           std
1749           1844     1.510118e+07  1.510118e+07  1.510118e+07           NaN
5468           5723     7.909083e+06  3.954542e+06  5.931812e+06  2.796283e+06
8811           9669     3.779820e+05  3.779820e+05  3.779820e+05           NaN
143             147     3.792198e+04  7.911200e+01  7.192822e+03  1.402930e+04
9769          12768     3.625200e+05  3.625200e+05  3.625200e+05           NaN

In [172]:
test_monto.sample(5)

Opportunity_ID monto_en_dolares                                   
                                 max         min        mean        std
804           11605       349387.500  349387.500  349387.500        NaN
365           11054         5093.550    2827.500    3940.125  1257.8866
438           11238        67795.533   67795.533   67795.533        NaN
1237          12039        31773.000   31773.000   31773.000        NaN
1235          12037        90394.044   90394.044   90394.044        NaN

In [173]:
train_monto.columns

MultiIndex([(  'Opportunity_ID',     ''),
            ('monto_en_dolares',  'max'),
            ('monto_en_dolares',  'min'),
            ('monto_en_dolares', 'mean'),
            ('monto_en_dolares',  'std')],
           )

In [174]:
train_monto.columns = ['Opportunity_ID','monto_en_dolares_max','monto_en_dolares_min','monto_en_dolares_mean','monto_en_dolares_std']
test_monto.columns = ['Opportunity_ID','monto_en_dolares_max','monto_en_dolares_min','monto_en_dolares_mean','monto_en_dolares_std']

In [175]:
train=pd.merge(train,train_monto,how='inner',on='Opportunity_ID')
test=pd.merge(test,test_monto,how='inner',on='Opportunity_ID')

### Stage

In [178]:
train.Stage.value_counts()

Closed Won       9529
Closed Lost      7315
Proposal           35
Negotiation        18
Qualification      11
Name: Stage, dtype: int64

* En el set de datos de *train* se observa que en la variable Stage, además de los estados que indican si una oportunidad resultó exitosa o no (**estados finales**), tenemos otros 3 estados (Proposal, Negotiation y Qualification) que corresponden a **estados intermedios** que tienen las oportunidades dentro del pipeline que desarrolla la empresa.

* Como con esta variable se estará luego tratando de predecir la “Probabilidad de éxito” para cada oportunidad en el set de *test*, o sea ¿Cuál es la probabilidad de que la oportunidad se convierta en un caso Closed Won? . Que decisión debemos tomar con dichos estados? Dichos estados al no tener una resolución, debemos asignarle arbitrariamente algún estado final? Debemos filtrar estos casos?

> * Finalmente luego de analizar esos registros con estos estados intermedios y verificar que corresponen a oportunidades que no tuvieron aún un desenlase (exitoso o no exitoso) y tenindo en cuenta que corresponden a unos pocos casos, tomamos la decisión de filtrar estos registros y quedarnos solamente con los que tienen un estado final definido.

In [179]:
train=train.loc[(train['Stage']!='Proposal') & (train['Stage']!='Negotiation') & (train['Stage']!='Qualification'),:]

In [180]:
train.Stage.value_counts()

Closed Won     9529
Closed Lost    7315
Name: Stage, dtype: int64

### Cantidad de Items de cada Oportunidad

In [181]:
Op_cant_train = train.groupby(['Opportunity_ID'], as_index=False)['ID'].count()
Op_cant_test = test.groupby(['Opportunity_ID'], as_index=False)['ID'].count()
Op_cant_train.rename(columns = {'ID': 'Cantidad_de_Items_Asociados'}, inplace = True)
Op_cant_test.rename(columns = {'ID': 'Cantidad_de_Items_Asociados'}, inplace = True)

train=pd.merge(Op_cant_train,train,on='Opportunity_ID')
test=pd.merge(Op_cant_test,test,on='Opportunity_ID')

In [182]:
train[['ID','Region','Opportunity_ID','Cantidad_de_Items_Asociados']].sample(5)

,ID,Region,Opportunity_ID,Cantidad_de_Items_Asociados
214,23906,Europa,115,1
6323,17639,América,3928,1
16661,25098,Asia-Pacífico,12716,7
212,19004,Europa,113,1
10350,18156,Asia-Pacífico,6500,2


In [183]:
test[['ID','Region','Opportunity_ID','Cantidad_de_Items_Asociados']].sample(5)

,ID,Region,Opportunity_ID,Cantidad_de_Items_Asociados
1936,22737,América,11992,3
1344,7607,América,11620,1
974,12818,Asia-Pacífico,11388,1
432,6206,Europa,10948,1
1894,10652,América,11970,2


### Precio total en dolares por Items de la Oportunidad

In [184]:
train['precio_por_items'] = train['total_en_dolares']/train['Cantidad_de_Items_Asociados']

In [185]:
test['precio_por_items'] = test['total_en_dolares']/test['Cantidad_de_Items_Asociados']

### Media del Total Amount en dolares por items de Oportunidad

In [187]:
train['media_por_items'] = train['monto_en_dolares_mean']/train['Cantidad_de_Items_Asociados']
test['media_por_items'] = test['monto_en_dolares_mean']/test['Cantidad_de_Items_Asociados']
#train['TRF_por_items'] = train['Cantidad de Items Asociados']*train['TRF']
#test['TRF_por_items'] = test['Cantidad de Items Asociados']*test['TRF']

### Calculamos la frecuencia de aparición para los valores coreespondientes a columnas categóricas para las distintas oportunidades

In [188]:
df = pd.concat([train,test])
df.columns

Index(['Opportunity_ID', 'Cantidad_de_Items_Asociados', 'ID', 'Region',
       'Territory', 'Pricing, Delivery_Terms_Quote_Appr',
       'Pricing, Delivery_Terms_Approved', 'Bureaucratic_Code_0_Approval',
       'Bureaucratic_Code_0_Approved', 'Submitted_for_Approval',
       'Bureaucratic_Code', 'Account_Created_Date', 'Source ',
       'Billing_Country', 'Account_Name', 'Opportunity_Name',
       'Sales_Contract_No', 'Account_Owner', 'Opportunity_Owner',
       'Account_Type', 'Opportunity_Type', 'Quote_Type', 'Delivery_Terms',
       'Opportunity_Created_Date', 'Brand', 'Product_Type', 'Size',
       'Product_Category_B', 'Price', 'Currency', 'Last_Activity',
       'Quote_Expiry_Date', 'Last_Modified_Date', 'Last_Modified_By',
       'Product_Family', 'Product_Name', 'ASP_Currency', 'ASP',
       'ASP_(converted)_Currency', 'ASP_(converted)',
       'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date', 'Month',
       'Delivery_Quarter', 'Delivery_Year', 'Actual_Delivery_Date

In [189]:
#Limpiamos los nombres de las columnas para que contengan solo valores alfanuméricos
import re
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [190]:
mis_brand=pd.crosstab(df.Opportunity_ID,df.Brand).reset_index()
df=pd.merge(df, mis_brand,on='Opportunity_ID' ,how='inner')

In [191]:
mis_Burocratic_code=pd.crosstab(df.Opportunity_ID,df.Bureaucratic_Code).reset_index()
df=pd.merge(df, mis_Burocratic_code,on='Opportunity_ID' ,how='inner')

In [192]:
mis_Product_Name=pd.crosstab(df.Opportunity_ID,df.Product_Name).reset_index()
df=pd.merge(df, mis_Product_Name,on='Opportunity_ID' ,how='inner')

In [193]:
mis_Opportunity_Owner=pd.crosstab(df.Opportunity_ID,df.Opportunity_Owner).reset_index()
df=pd.merge(df, mis_Opportunity_Owner,on='Opportunity_ID' ,how='inner')

In [194]:
mis_Account_Owner=pd.crosstab(df.Opportunity_ID,df.Account_Owner).reset_index()
df=pd.merge(df, mis_Account_Owner,on='Opportunity_ID' ,how='inner')

In [195]:
mis_Product_Family=pd.crosstab(df.Opportunity_ID,df.Product_Family).reset_index()
df=pd.merge(df, mis_Product_Family,on='Opportunity_ID' ,how='inner')

In [196]:
mis_Account_Name=pd.crosstab(df.Opportunity_ID,df.Account_Name).reset_index()
df=pd.merge(df, mis_Account_Name,on='Opportunity_ID' ,how='inner')

In [197]:
#mis_Estacin=pd.crosstab(df.Opportunity_ID,df.Estacin).reset_index()
#df=pd.merge(df, mis_Estacin,on='Opportunity_ID' ,how='inner')

In [198]:
mis_Estacion_entrega=pd.crosstab(df.Opportunity_ID,df.Estacion_entrega).reset_index()
df=pd.merge(df, mis_Estacion_entrega,on='Opportunity_ID' ,how='inner')

In [199]:
mis_Source=pd.crosstab(df.Opportunity_ID,df.Source).reset_index()
df=pd.merge(df, mis_Source,on='Opportunity_ID' ,how='inner')

### Fechas

In [200]:
df['Planned_Delivery_Start_Date']=pd.to_datetime(df['Planned_Delivery_Start_Date'])
df['Planned_Delivery_End_Date']=pd.to_datetime(df['Planned_Delivery_End_Date'])
df['year_entrega']=df['Planned_Delivery_End_Date'].dt.year
df['mes_entrega']=df['Planned_Delivery_End_Date'].dt.month
df['Opportunity_Created_Date']=pd.to_datetime(df['Opportunity_Created_Date'])
df['Last_Activity']=pd.to_datetime(df['Last_Activity'])
df['Account_Created_Date']=pd.to_datetime(df['Account_Created_Date'])
df['Quote_Expiry_Date']=pd.to_datetime(df['Quote_Expiry_Date'])
df['Last_Modified_Date']=pd.to_datetime(df['Last_Modified_Date'])
#df.loc[df['Quote_Expiry_Date'].isnull(),'Quote_Expiry_Date']=pd.datetime.now()
#df.loc[df['Quote_Expiry_Date'].isnull(),'Quote_Expiry_Date']=pd.datetime(2014, 1, 1)
#df['Quote_Expiry_Date']=pd.to_datetime(df['Quote_Expiry_Date'])

In [202]:
df['tiempo_plan_end_star']=df['Planned_Delivery_End_Date']-df['Planned_Delivery_Start_Date']
df['tiempo_plan_end_star'] = df['tiempo_plan_end_star'].dt.total_seconds()/36000
df.fillna(0,inplace=True)
df['tiempo_entrega_por_items'] = df['tiempo_plan_end_star']/df['Cantidad_de_Items_Asociados']

#df['tiempo_cuenta_oportunidad']=df['Account_Created_Date']-df['Opportunity_Created_Date']
#df['tiempo_cuenta_oportunidad']=df['tiempo_cuenta_oportunidad'].dt.total_seconds()/36000

#df['tiempo_presupuesto_entrega']=df['Planned_Delivery_Start_Date']-df['Quote_Expiry_Date']
#df['tiempo_presupuesto_entrega']=df['tiempo_presupuesto_entrega'].dt.total_seconds()/360000

#df['tiempo_modificacion_creacion']=df['Opportunity_Created_Date']-df['Last_Modified_Date']
#df['tiempo_modificacion_creacion']=df['tiempo_modificacion_creacion'].dt.total_seconds()/36000
#df['year_entrega']=df['Planned_Delivery_Start_Date'].dt.year
#df['mes_entrega']=df['Planned_Delivery_Start_Date'].dt.month

In [203]:
train =train[['ID']]
train = pd.merge(train,df, on= 'ID', how = 'left') 
test =test[['ID']]
test = pd.merge(test,df, on= 'ID', how = 'left')

In [204]:
train.head()

,ID,Opportunity_ID,Cantidad_de_Items_Asociados,Region,Territory,PricingDelivery_Terms_Quote_Appr,PricingDelivery_Terms_Approved,Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,...,Source_3,Source_4,Source_5,Source_6,Source_7,Source_9,year_entrega,mes_entrega,tiempo_plan_end_star,tiempo_entrega_por_items
0,27761,0,1,Europa,None,1,1,1,1,0,...,0,0,0,0,0,0,2016.0,6.0,144.0,144.0
1,27760,1,1,Europa,None,0,0,0,0,0,...,0,0,0,0,0,0,2016.0,1.0,4.8,4.8
2,27446,2,1,América,NW America,0,0,0,0,0,...,0,0,0,0,1,0,2016.0,1.0,0.0,0.0
3,16808,3,1,América,NW America,1,0,1,0,0,...,0,0,0,0,0,0,2018.0,3.0,139.2,139.2
4,16805,4,1,América,NW America,1,0,1,0,0,...,0,0,0,0,0,0,2018.0,2.0,64.8,64.8


In [205]:
train_tiempo_entrega=train.groupby('Opportunity_ID').agg({'tiempo_plan_end_star':['max','min','mean','std']}).reset_index()
train_tiempo_entrega.columns = ['Opportunity_ID','tiempo_entrega_max','tiempo_entrega_min','tiempo_entrega_mean','tiempo_entrega_std']
test_tiempo_entrega=test.groupby('Opportunity_ID').agg({'tiempo_plan_end_star':['max','min','mean','std']}).reset_index()
test_tiempo_entrega.columns = ['Opportunity_ID','tiempo_entrega_max','tiempo_entrega_min','tiempo_entrega_mean','tiempo_entrega_std']

In [206]:
train=pd.merge(train,train_tiempo_entrega,how='inner',on='Opportunity_ID')
test=pd.merge(test,test_tiempo_entrega,how='inner',on='Opportunity_ID')

In [207]:
train.to_csv("train_listo.csv", index=False)
test.to_csv("test_listo.csv", index=False)